In [1]:
import pandas as pd

# Load the dataset
df = pd.read_excel('Pvt Car Jan to  Nov 24.xlsx')

In [2]:
# Convert relevant columns to datetime format
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'])
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'])

# Identify duplicates based on 'Policy Number', 'Policy Start Date', and 'Policy End Date'
duplicates = df[df.duplicated(subset=['Policy Number', 'Policy Start Date', 'Policy End Date'], keep=False)]

# Remove duplicates, keeping only one row per duplicate group
df_cleaned = df.drop_duplicates(subset=['Policy Number', 'Policy Start Date', 'Policy End Date'], keep='first')

# Identify rows that were removed
removed_rows = pd.concat([df, df_cleaned]).drop_duplicates(keep=False)

removed_rows.to_csv('duplicates_data (2024 Base).csv', index=False)

df_cleaned.to_csv('cleaned_2024_Base_dataset.csv', index=False)

In [3]:
import pandas as pd
import re

df = pd.read_excel('cleaned_2024_Base_dataset.xlsx')

In [4]:
# Function to clean names
def clean_name(name):
    return re.sub(r'[^a-zA-Z0-9]', '', str(name)).lower()

# Create a new column with the cleaned INSUREDNAME
df['Cleaned_Insured name'] = df['Insured name '].apply(clean_name)

df['CustomerID_Base'] = (df['Cleaned_Insured name'].astype(str) + '_' +
                         df['New Branch Name  2'].astype(str))

df['CustomerID'] = (df.groupby('CustomerID_Base').ngroup() + 1).astype(str).str.zfill(5)

df.to_csv('cleanedname_2024_Base_dataset_data.csv', index=False)

print(df[['Insured name ', 'Cleaned_Insured name', 'CustomerID']].head())

                     Insured name            Cleaned_Insured name CustomerID
0        VIPULKUMAR LAXMIDAS TANNA        vipulkumarlaxmidastanna     594382
1                        LAMIN ART                       laminart     249334
2  DEVENDRAKUMAR NITINBHAI VAGHELA  devendrakumarnitinbhaivaghela     119309
3                        MADAN LAL                       madanlal     255937
4                     JASBIR SINGH                    jasbirsingh     192740


In [12]:
import pandas as pd
from datetime import timedelta

# Placeholder DataFrame: Replace this with actual data loading
df = pd.read_csv("cleanedname_2024_Base_dataset_data.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6076\355171939.py:5: DtypeWarning: Columns (42,43,50,52,53,55) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleanedname_2024_Base_dataset_data.csv")


In [13]:
# Convert 'Policy Start Date' and 'Policy End Date' to datetime format
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Extract the year from start and end dates
df['Start_Year'] = df['Policy Start Date'].dt.year
df['End_Year'] = df['Policy End Date'].dt.year

# Identify year-wise new customers
df['FirstPolicyYear'] = df.groupby('CustomerID')['Start_Year'].transform('min')

# Generate the year-wise new customer column
df['New_Customer_ID'] = df.apply(
    lambda row: f"{row['FirstPolicyYear']}_{row['CustomerID']}" if row['Start_Year'] == row['FirstPolicyYear'] else '',
    axis=1
)

# Save the updated dataset to a new CSV file
df.to_csv("updated_cleanedname_2024_Base_dataset_data.csv", index=False)

In [15]:
import pandas as pd

df = pd.read_excel('updated_cleanedname_2024_Base_dataset_data.xlsx')

In [16]:
import pandas as pd

# Ensure proper datetime formatting
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy End Date'] = pd.to_datetime(df['Policy End Date'], errors='coerce')

# Extract Start Year and Year-Month
df['Start Year'] = df['Policy Start Date'].dt.year
df['Start Year-Month'] = df['Policy Start Date'].dt.to_period('M')
df['End Year-Month'] = df['Policy End Date'].dt.to_period('M')

# 1. Start Year wise customer counts and percentages
start_year_metrics = (
    df.groupby('Start Year')
    .apply(lambda x: pd.Series({
        'Total Customers': x['CustomerID'].nunique(),
        'New Customers': x[x['New Customer'] == 'Yes']['CustomerID'].nunique(),
        'Old Customers': x[x['New Customer'] == 'No']['CustomerID'].nunique()
    }))
    .reset_index()
)

start_year_metrics['New Customer %'] = (start_year_metrics['New Customers'] / start_year_metrics['Total Customers']) * 100
start_year_metrics['Old Customer %'] = (start_year_metrics['Old Customers'] / start_year_metrics['Total Customers']) * 100

# 2. Start Date year-month-wise customer counts and percentages
start_year_month_metrics = (
    df.groupby('Start Year-Month')
    .apply(lambda x: pd.Series({
        'Total Customers': x['CustomerID'].nunique(),
        'New Customers': x[x['New Customer'] == 'Yes']['CustomerID'].nunique(),
        'Old Customers': x[x['New Customer'] == 'No']['CustomerID'].nunique()
    }))
    .reset_index()
)

start_year_month_metrics['New Customer %'] = (start_year_month_metrics['New Customers'] / start_year_month_metrics['Total Customers']) * 100
start_year_month_metrics['Old Customer %'] = (start_year_month_metrics['Old Customers'] / start_year_month_metrics['Total Customers']) * 100

# Save metrics to an Excel file
file_path = 'Year_and_Year-Month_Metrics (2024 Base).xlsx'
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
    start_year_metrics.to_excel(writer, index=False, sheet_name='Start Year Metrics')
    start_year_month_metrics.to_excel(writer, index=False, sheet_name='Start Year-Month Metrics')

print(f"Metrics saved to {file_path}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6076\3929638163.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
C:\Users\Admin\AppData\Local\Temp\ipykernel_6076\3929638163.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


Metrics saved to Year_and_Year-Month_Metrics (2024 Base).xlsx
